In [1]:
# 导入包
from docplex.mp.model import Model
import pandas as pd

# 建立模型
m = Model(name='experimentmodel1.3.3')

In [2]:
# 定义参数
n_city = 9

# 城市索引
set_city = range(1, n_city + 1)

# 各城市之间的距离
distance = pd.read_csv("min_distance.csv", usecols=[i for i in set_city])
distance.columns = [i for i in set_city]
distance.index = [i for i in set_city]

# 各城市建设配送中心的成本
cost = pd.Series(pd.read_csv("cost.csv")['cost'].values, index = [i for i in set_city])

# 可覆盖关系矩阵
coverable_distance = 300
cover_matrix = pd.DataFrame(columns=[i for i in set_city], index = [i for i in set_city])
for i in set_city:
    for j in set_city:
        if distance.loc[i,j] <= coverable_distance:
            cover_matrix.loc[i,j] = True
        else:
            cover_matrix.loc[i,j] = False
cover_matrix

,1,2,3,4,5,6,7,8,9
1,True,True,True,True,False,False,False,False,False
2,True,True,True,False,False,False,False,False,False
3,True,True,True,True,True,False,False,False,False
4,True,False,True,True,True,True,True,False,False
5,False,False,True,True,True,True,False,False,False
6,False,False,False,True,True,True,True,True,False
7,False,False,False,True,False,True,True,True,False
8,False,False,False,False,False,True,True,True,True
9,False,False,False,False,False,False,False,True,True


In [3]:
# 定义变量
x_vars = pd.Series([m.binary_var(name="x_{0}".format(j)) for j in set_city])
x_vars.index = [j for j in set_city]

In [4]:
# 定义约束条件
distance_constrains = {i : m.add_constraint(
    sum(cover_matrix.loc[i,j] * x_vars.loc[j] for j in set_city) >= 1,
    ctname="distance_constrain_{0}".format(i))
              for i in set_city}
capability_constrain = m.add_constraint(x_vars.loc[6] == 0)

In [5]:
# 定义目标函数
# 建设总成本最小
objective = sum(cost.loc[j] * x_vars.loc[j] for j in set_city)
# 配送中心数量最小
# objective = sum(x_vars.loc[j] for j in set_city)

# for minimization             
m.minimize(objective)

In [6]:
# 打印模型信息
m.print_information()

Model: experimentmodel1.3.3
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 10
   - linear=10
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [7]:
# 模型求解及结果输出
s = m.solve(log_output=True)
m.print_solution()

Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 25.000000 after 0.01 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 8 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.02 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.02 ticks)
objective: 10
  x_3=1
  x_8=1


In [8]:
# 输出结果
x_vars = x_vars.apply(lambda item: item.solution_value) 
x_vars.to_csv("solution_x.csv")

In [9]:
x_vars

1    0.0
2    0.0
3    1.0
4    0.0
5    0.0
6    0.0
7    0.0
8    1.0
9    0.0
dtype: float64